<a href="https://colab.research.google.com/github/poorvitiwary/Computer_vision_Age/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.15.2


In [6]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import  GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [7]:
!ls data

'middle age'  'new born'   old	'young adult'


In [8]:
HP_dataset = 'data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dim = (96,96,3)

In [9]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset)))
all_images[:5]

['data/middle age/middleage_0.png',
 'data/middle age/middleage_1.png',
 'data/middle age/middleage_10.png',
 'data/middle age/middleage_100.png',
 'data/middle age/middleage_101.png']

In [10]:
random.seed(42)
random.shuffle(all_images)
all_images[:5]


['data/old/oldage_328.png',
 'data/middle age/middleage_146.png',
 'data/new born/newborn_190.png',
 'data/young adult/youngadult_71.png',
 'data/old/oldage_255.png']

In [11]:
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)

In [12]:
data = np.array(data, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0]])

In [13]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(data, labels, test_size=0.25, random_state=42)


In [14]:
#Augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.2, 
                         height_shift_range=0.2, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [ ]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth,classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1

   # model = Sequential()
    cnn1 = Sequential()

    cnn1.add(Conv2D(filters=8, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))    # 3rd dim = 1 for grayscale images.
    cnn1.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
    cnn1.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
    cnn1.add(MaxPooling2D(pool_size=(2,2)))

    cnn1.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
    cnn1.add(MaxPooling2D(pool_size=(2,2)))

    cnn1.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
    cnn1.add(MaxPooling2D(pool_size=(2,2)))

    cnn1.add(GlobalAveragePooling2D())
    cnn1.add(Dense(20, activation='relu'))
    cnn1.add(Dense(11, activation='softmax'))


In [25]:
cnn1.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

NameError: ignored

In [ ]:
len(trainx)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [ ]:
history = cnn1.fit(trainx, trainy, epochs=10, 
                    validation_data=(testx, testy),
                    callbacks = [early_stop])